## Prerequisites - Environment Setup

Before starting this workshop, you need to set up a .NET development environment. Follow these steps:

### Step 1: Install .NET 9 SDK
Download and install the latest .NET 9 SDK from [https://dotnet.microsoft.com/download/dotnet/9.0](https://dotnet.microsoft.com/download/dotnet/9.0)

### Step 2: Install .NET Interactive Extension
In VS Code, install the ".NET Interactive Notebooks" extension to run C# code in Jupyter notebooks.

### Step 3: Create Project Directory
From the **root folder** of this repository, ensure you have the proper project structure:
```bash
Backend/dotnet/sk/
```

### Step 4: Create .env Configuration File
Create a `.env` file in this directory (or the parent directory) with your Azure OpenAI credentials:
```bash
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini
```

**Important:** Never commit your `.env` file to version control! Add it to `.gitignore`.

### Step 5: Verify .NET Installation
Run the following command to verify your .NET installation:
```bash
dotnet --version
```

### Step 6: Select .NET Interactive Kernel
1. Click on the **kernel selector** in the top-right corner of this notebook
2. Select **".NET Interactive"**
3. Choose **"C#"** as the language

### Step 7: Verify Setup
Once you've selected the correct kernel and created your .env file, you can proceed with the workshop. The first code cell will verify your .NET environment.

---

# .NET Semantic Kernel Agents Workshop

This notebook demonstrates how to build AI agents using Microsoft Semantic Kernel in .NET 9 with Azure OpenAI services, following the same patterns as the Python workshop.

## Workshop Overview
1. **Environment Setup** - Verify .NET and Semantic Kernel installation
2. **Azure OpenAI Configuration** - Automatically load from .env file
3. **Basic Agent Creation** - Create simple conversational agents
4. **Specialized Agents** - Build agents with specific roles and expertise
5. **Group Chat System** - Multi-agent collaboration with orchestration
6. **Advanced Features** - Memory and production considerations
7. **Integration Demo** - Connect with the .NET API backend

Let's build sophisticated AI agent systems using the power of .NET and Semantic Kernel!

### Step 1: Verify .NET Environment

In [ ]:
// Step 1: Verify .NET Environment
// Check if .NET Interactive is working properly

using System;
using System.Runtime.InteropServices;

Console.WriteLine("=== .NET Environment Check ===");
Console.WriteLine($".NET Version: {Environment.Version}");
Console.WriteLine($"Runtime: {RuntimeInformation.FrameworkDescription}");
Console.WriteLine($"OS: {RuntimeInformation.OSDescription}");
Console.WriteLine($"Architecture: {RuntimeInformation.OSArchitecture}");
Console.WriteLine();
Console.WriteLine("✓ .NET Interactive is working!");
Console.WriteLine("Ready to install Semantic Kernel packages!");

// Return a value to ensure cell execution is visible
Console.WriteLine("Environment check completed successfully!");
"SUCCESS: .NET Interactive is running properly"

### Step 2: Install Semantic Kernel Packages

In [ ]:
// Step 2: Install Required NuGet Packages
// Install the latest Semantic Kernel packages for agent development

Console.WriteLine("Installing Semantic Kernel packages...");

// Core Semantic Kernel package
#r "nuget: Microsoft.SemanticKernel, 1.30.0"

// Agents package for multi-agent scenarios  
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.30.0-alpha"

// Azure OpenAI connector
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.30.0"

// .NET Configuration and Environment support
#r "nuget: DotNetEnv, 3.1.1"

// Additional useful packages
#r "nuget: Microsoft.Extensions.Logging, 9.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 9.0.0"
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"

Console.WriteLine("✓ All Semantic Kernel packages installed successfully!");
Console.WriteLine("Ready to import namespaces and create agents!")

### Troubleshooting: If You Don't See Output

**If the Run button does nothing or no output appears:**

#### 🚨 Most Common Issue: Kernel Not Selected
1. **Look at the top-right corner** of the notebook - you should see something like ".NET Interactive" or "C#"
2. **If you see "Select Kernel"** or a different kernel:
   - Click on it
   - Select **".NET Interactive"**
   - Choose **"C#"** from the sub-menu

#### 🔧 Other Solutions to Try:

1. **Install .NET Interactive Extension**:
   - Press `Ctrl+Shift+X` to open Extensions
   - Search for **".NET Interactive Notebooks"** by Microsoft
   - Install if not already installed
   - **Reload VS Code** after installation

2. **Restart Everything**:
   - Press `Ctrl+Shift+P`
   - Type "Developer: Reload Window"
   - Reopen the notebook and try again

3. **Check .NET Installation**:
   - Open a terminal (`Ctrl+Shift+``)
   - Run: `dotnet --version`
   - Should show .NET 8.0 or later

4. **Force Kernel Selection**:
   - Press `Ctrl+Shift+P`
   - Type "Notebook: Select Notebook Kernel"
   - Choose ".NET Interactive" → "C#"

**The cell below should show "42" as output if everything is working.**

### Step 3: Automatic .env Configuration

This notebook automatically loads your Azure OpenAI configuration from a `.env` file, making it much easier for beginners! 

**How it works:**
1. The notebook searches for a `.env` file in the current directory or parent directories
2. It automatically loads all environment variables from the file
3. No need to manually set environment variables or edit code

**Sample .env file:**
```
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini
```

**Benefits of .env approach:**
- ✅ No code modification needed
- ✅ Secure credential management
- ✅ Easy to share configurations
- ✅ Matches Python workshop pattern
- ✅ Beginner-friendly setup

**Next:** Import Required Namespaces

In [ ]:
// Step 3: Import Required Namespaces
// Import all the Semantic Kernel namespaces we'll use for our agents

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;
using System.ComponentModel;
using System.IO;
using DotNetEnv;

Console.WriteLine("✓ All namespaces imported successfully!");
Console.WriteLine("Ready to configure Azure OpenAI!");

### Step 4: Configure Azure OpenAI

In [ ]:
// Step 4: Configure Azure OpenAI Connection
// Automatically load configuration from .env file

Console.WriteLine("Configuring Azure OpenAI...");

// Load .env file from current directory or parent directories
try
{
    // Try loading .env from current notebook directory
    var notebookDir = Directory.GetCurrentDirectory();
    var envFile = Path.Combine(notebookDir, ".env");
    
    if (File.Exists(envFile))
    {
        Env.Load(envFile);
        Console.WriteLine($"✓ Loaded .env file from: {envFile}");
    }
    else
    {
        // Try parent directories (common pattern)
        var parentDir = Directory.GetParent(notebookDir);
        while (parentDir != null)
        {
            envFile = Path.Combine(parentDir.FullName, ".env");
            if (File.Exists(envFile))
            {
                Env.Load(envFile);
                Console.WriteLine($"✓ Loaded .env file from: {envFile}");
                break;
            }
            parentDir = parentDir.Parent;
        }
        
        if (parentDir == null)
        {
            Console.WriteLine("! No .env file found, using system environment variables");
        }
    }
}
catch (Exception ex)
{
    Console.WriteLine($"! Error loading .env file: {ex.Message}");
    Console.WriteLine("Falling back to system environment variables");
}

// Load configuration from environment variables (now populated from .env)
var azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var azureOpenAIApiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME") ?? "gpt-4o-mini";

// Required environment variables
var requiredVars = new Dictionary<string, string?>
{
    { "AZURE_OPENAI_ENDPOINT", azureOpenAIEndpoint },
    { "AZURE_OPENAI_API_KEY", azureOpenAIApiKey },
    { "AZURE_OPENAI_DEPLOYMENT_NAME", deploymentName }
};

// Check if all required variables are present
var missingVars = requiredVars.Where(kvp => string.IsNullOrWhiteSpace(kvp.Value)).Select(kvp => kvp.Key).ToList();

if (missingVars.Any())
{
    Console.WriteLine("❌ Missing required Azure OpenAI configuration!");
    Console.WriteLine($"Missing variables: {string.Join(", ", missingVars)}");
    Console.WriteLine();
    Console.WriteLine("To fix this, create a .env file in this directory (or parent directory) with:");
    Console.WriteLine("AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/");
    Console.WriteLine("AZURE_OPENAI_API_KEY=your_api_key_here");
    Console.WriteLine("AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini");
    Console.WriteLine();
    Console.WriteLine("Then restart this notebook kernel and run this cell again.");
    throw new InvalidOperationException("Required Azure OpenAI configuration missing!");
}

Console.WriteLine("✓ Azure OpenAI configuration loaded successfully!");
Console.WriteLine($"Endpoint: {azureOpenAIEndpoint}");
Console.WriteLine($"Deployment: {deploymentName}");
Console.WriteLine("Ready to create Semantic Kernel!");

### Step 5: Create Semantic Kernel Instance

In [ ]:
// Step 5: Create Semantic Kernel Instance
// Set up the Semantic Kernel with Azure OpenAI chat completion

Console.WriteLine("Creating Semantic Kernel...");

// Create Kernel Builder
var builder = Kernel.CreateBuilder();

// Add Azure OpenAI Chat Completion service
builder.AddAzureOpenAIChatCompletion(
    deploymentName: deploymentName,
    endpoint: azureOpenAIEndpoint,
    apiKey: azureOpenAIApiKey);

// Add logging
builder.Services.AddLogging(c => c.AddConsole().SetMinimumLevel(LogLevel.Information));

// Build the kernel
var kernel = builder.Build();

Console.WriteLine("✓ Semantic Kernel created successfully!");
Console.WriteLine("Ready to create our first agent!");

In [ ]:
// Step 6: Create Your First Semantic Kernel Agent
// This creates a generic agent class similar to the Python implementation

public class GenericAgent
{
    public string Name { get; set; }
    public string Description { get; set; }
    public Kernel Kernel { get; set; }
    
    public GenericAgent(string name, string description, string? deploymentName = null)
    {
        Name = name;
        Description = description;
        
        // Use deployment name from environment if not provided
        deploymentName ??= Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME") ?? "gpt-4o-mini";
        
        // Get Azure OpenAI configuration from environment variables (loaded from .env)
        var endpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
        var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
        
        if (string.IsNullOrWhiteSpace(endpoint) || string.IsNullOrWhiteSpace(apiKey))
        {
            throw new InvalidOperationException("Azure OpenAI configuration is missing. Please ensure .env file is properly loaded.");
        }
        
        // Create a new kernel for this agent
        var builder = Kernel.CreateBuilder();
        
        // Add Azure OpenAI Chat Completion service
        builder.AddAzureOpenAIChatCompletion(
            deploymentName: deploymentName,
            endpoint: endpoint,
            apiKey: apiKey);
        
        // Add logging
        builder.Services.AddLogging(c => c.AddConsole().SetMinimumLevel(LogLevel.Information));
        
        Kernel = builder.Build();
    }
    
    public async Task<string> ChatAsync(string message)
    {
        // Create a system message with the agent's description
        var systemMessage = $"You are {Name}. {Description}";
        
        // Combine system message with user message
        var fullPrompt = $"{systemMessage}\n\nUser: {message}\nAssistant:";
        
        // Get response from the kernel
        var response = await Kernel.InvokePromptAsync(fullPrompt);
        return response.ToString();
    }
}

// Create our first generic agent
var genericAgent = new GenericAgent(
    "Assistant",
    "A helpful AI assistant that can answer questions and have conversations."
);

Console.WriteLine("✓ Generic Agent created successfully!");
Console.WriteLine($"Agent Name: {genericAgent.Name}");
Console.WriteLine($"Description: {genericAgent.Description}");
Console.WriteLine("Ready to test the agent!");

### Test Your First Agent

In [ ]:
// Test the Generic Agent
// Send a message to the agent and see its response

Console.WriteLine("Testing the generic agent...");

// Test message - you can modify this to ask different questions
var userMessage = "Hello! What is artificial intelligence?";

Console.WriteLine($"User: {userMessage}");

try
{
    // Get agent response using the new ChatAsync method
    var response = await genericAgent.ChatAsync(userMessage);
    Console.WriteLine($"Agent ({genericAgent.Name}): {response}");
    
    Console.WriteLine();
    Console.WriteLine("✓ Agent test completed successfully!");
    Console.WriteLine("You can modify the userMessage above to test different questions.");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error testing agent: {ex.Message}");
    Console.WriteLine("Please check your Azure OpenAI configuration.");
}

### Create Specialized Agents

In [ ]:
// Create Specialized Agents
// Build agents with specific roles and expertise, similar to the Python implementation

public class SpecializedAgent : GenericAgent
{
    public string Role { get; set; }
    public string Expertise { get; set; }
    public string Personality { get; set; }
    
    public SpecializedAgent(string name, string role, string expertise, string personality = "professional and helpful")
        : base(name, CreateDescription(name, role, expertise, personality))
    {
        Role = role;
        Expertise = expertise;
        Personality = personality;
    }
    
    private static string CreateDescription(string name, string role, string expertise, string personality)
    {
        return $@"You are {name}, a {role} with expertise in {expertise}. 
        Your personality is {personality}.
        
        Guidelines for responses:
        - Focus on your area of expertise: {expertise}
        - Maintain your role as a {role}
        - Be {personality} in all interactions
        - Provide practical, actionable advice when possible
        - If asked about topics outside your expertise, acknowledge the limitation but offer what help you can";
    }
}

Console.WriteLine("Creating specialized agents...");

// Technical Expert Agent
var techExpert = new SpecializedAgent(
    "TechExpert",
    "Senior Software Engineer",
    ".NET programming, software architecture, and best practices",
    "analytical and detail-oriented"
);

// Business Expert Agent
var businessExpert = new SpecializedAgent(
    "BusinessGuru",
    "Business Strategy Consultant",
    "market analysis, business strategy, and digital transformation",
    "strategic and results-focused"
);

// Creative Expert Agent  
var creativeExpert = new SpecializedAgent(
    "CreativeMind",
    "Creative Director",
    "creative writing, marketing campaigns, and content strategy",
    "imaginative and inspiring"
);

Console.WriteLine("✓ Specialized agents created successfully!");
Console.WriteLine($"1. {techExpert.Name} - {techExpert.Role}");
Console.WriteLine($"2. {businessExpert.Name} - {businessExpert.Role}");
Console.WriteLine($"3. {creativeExpert.Name} - {creativeExpert.Role}");
Console.WriteLine();
Console.WriteLine("Each agent has specialized knowledge and a unique personality!");

In [ ]:
// Test Specialized Agents  
// Let's test each specialized agent with questions in their area of expertise

Console.WriteLine("Testing specialized agents...");
Console.WriteLine();

// Test cases for each agent - matching the Python implementation
var testCases = new[]
{
    new { Agent = techExpert, Question = "What are the best practices for writing clean C# code?" },
    new { Agent = businessExpert, Question = "How should a startup approach digital transformation?" },
    new { Agent = creativeExpert, Question = "Create a catchy tagline for a new eco-friendly water bottle" }
};

for (int i = 0; i < testCases.Length; i++)
{
    var testCase = testCases[i];
    
    try
    {
        Console.WriteLine($"Test {i + 1}: {testCase.Agent.Name} ({testCase.Agent.Role})");
        Console.WriteLine($"Question: {testCase.Question}");
        Console.WriteLine(new string('-', 50));
        
        var response = await testCase.Agent.ChatAsync(testCase.Question);
        Console.WriteLine($"Response: {response}");
        
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error testing {testCase.Agent.Name}: {ex.Message}");
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
}

Console.WriteLine("✓ Specialized agent testing completed!");
Console.WriteLine("Notice how each agent responds according to their role and expertise!");

### Multi-Agent Group Chat

In [ ]:
// Multi-Agent Group Chat with Semantic Kernel
// Create a simpler orchestrator that coordinates multiple agents

public class GroupChatOrchestrator
{
    private readonly Dictionary<string, GenericAgent> _agents;
    private readonly Kernel _kernel;
    
    public GroupChatOrchestrator(params GenericAgent[] agents)
    {
        _agents = agents.ToDictionary(a => a.Name, a => a);
        
        // Get Azure OpenAI configuration from environment variables (loaded from .env)
        var endpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
        var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
        var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME") ?? "gpt-4o-mini";
        
        if (string.IsNullOrWhiteSpace(endpoint) || string.IsNullOrWhiteSpace(apiKey))
        {
            throw new InvalidOperationException("Azure OpenAI configuration is missing. Please ensure .env file is properly loaded.");
        }
        
        // Create orchestrator's own kernel for coordination
        var builder = Kernel.CreateBuilder();
        builder.AddAzureOpenAIChatCompletion(
            deploymentName: deploymentName,
            endpoint: endpoint,
            apiKey: apiKey);
        builder.Services.AddLogging(c => c.AddConsole().SetMinimumLevel(LogLevel.Information));
        _kernel = builder.Build();
    }
    
    public async Task<List<string>> SelectRelevantAgentsAsync(string question, int maxAgents = 3)
    {
        // Create agent descriptions for selection
        var agentDescriptions = _agents.Values
            .Select(a => $"- {a.Name}: {(a is SpecializedAgent sa ? $"{sa.Role} with expertise in {sa.Expertise}" : a.Description)}")
            .ToList();
        
        var agentList = string.Join("\n", agentDescriptions);
        
        var selectionPrompt = $@"
        Given this question: ""{question}""
        
        Available agents:
        {agentList}
        
        Select the {maxAgents} most relevant agents to answer this question. 
        Consider which agents' expertise would be most valuable.
        
        Respond with ONLY the agent names, separated by commas (e.g., ""TechExpert, BusinessGuru"").
        ";
        
        try
        {
            var response = await _kernel.InvokePromptAsync(selectionPrompt.Trim());
            var selectedNames = response.ToString().Split(',')
                .Select(name => name.Trim())
                .Where(name => _agents.ContainsKey(name))
                .Take(maxAgents)
                .ToList();
            
            // If no valid names, default to first available agents
            if (!selectedNames.Any())
                selectedNames = _agents.Keys.Take(maxAgents).ToList();
                
            return selectedNames;
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error selecting agents: {ex.Message}");
            return _agents.Keys.Take(maxAgents).ToList();
        }
    }
    
    public async Task<string> GroupChatAsync(string question, int maxAgents = 3)
    {
        Console.WriteLine($"Question: {question}");
        Console.WriteLine(new string('=', 60));
        
        // Select relevant agents
        Console.WriteLine("Selecting relevant agents...");
        var selectedAgentNames = await SelectRelevantAgentsAsync(question, maxAgents);
        
        Console.WriteLine($"Selected agents: {string.Join(", ", selectedAgentNames)}");
        Console.WriteLine(new string('-', 60));
        
        // Collect responses from selected agents
        var agentResponses = new Dictionary<string, string>();
        
        foreach (var agentName in selectedAgentNames)
        {
            if (_agents.TryGetValue(agentName, out var agent))
            {
                try
                {
                    Console.WriteLine($"\nGetting response from {agentName}...");
                    var response = await agent.ChatAsync(question);
                    agentResponses[agentName] = response;
                    
                    // Display the response
                    var roleInfo = agent is SpecializedAgent sa ? $" ({sa.Role})" : "";
                    Console.WriteLine($"{agentName}{roleInfo}:");
                    Console.WriteLine(response);
                    Console.WriteLine(new string('-', 40));
                }
                catch (Exception ex)
                {
                    Console.WriteLine($"Error getting response from {agentName}: {ex.Message}");
                    agentResponses[agentName] = $"Error: {ex.Message}";
                }
            }
        }
        
        // Create synthesis
        var responseSummary = agentResponses.Select(kvp => 
        {
            var agent = _agents[kvp.Key];
            var role = agent is SpecializedAgent sa ? sa.Role : "Agent";
            return $"**{kvp.Key} ({role})**: {kvp.Value}";
        });
        
        var synthesisPrompt = $@"
        Question: {question}
        
        Expert responses:
        {string.Join("\n", responseSummary)}
        
        Please synthesize these expert opinions into a comprehensive, well-structured answer.
        Combine the different perspectives and highlight key insights from each expert.
        Make the final answer cohesive and actionable.
        ";
        
        try
        {
            Console.WriteLine("\nSynthesizing group response...");
            var synthesis = await _kernel.InvokePromptAsync(synthesisPrompt.Trim());
            
            Console.WriteLine("\n" + new string('=', 60));
            Console.WriteLine("SYNTHESIZED GROUP RESPONSE:");
            Console.WriteLine(new string('=', 60));
            Console.WriteLine(synthesis.ToString());
            
            return synthesis.ToString();
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error creating synthesis: {ex.Message}");
            return "Error creating synthesis. Please review individual agent responses above.";
        }
    }
}

// Create the group chat orchestrator with our specialized agents
var orchestrator = new GroupChatOrchestrator(
    genericAgent,
    techExpert, 
    businessExpert,
    creativeExpert
);

Console.WriteLine("✓ Group Chat Orchestrator created successfully!");
Console.WriteLine($"Available agents: {string.Join(", ", new[] { genericAgent.Name, techExpert.Name, businessExpert.Name, creativeExpert.Name })}");
Console.WriteLine("Ready for group collaboration!");

### Test Group Chat System

In [ ]:
// Test Group Chat - Product Launch Strategy
// Watch how multiple agents collaborate to create a comprehensive product launch strategy

Console.WriteLine("Testing Group Chat with Product Launch Scenario");
Console.WriteLine("This will demonstrate how multiple agents collaborate...");
Console.WriteLine();

// Complex question that benefits from multiple perspectives - matching Python version
var complexQuestion = @"We're launching a new AI-powered mobile app that helps people learn languages. 
We need a complete strategy covering technical architecture, business model, 
and marketing approach. What should we consider for a successful launch?";

try
{
    // Run the group chat
    var result = await orchestrator.GroupChatAsync(complexQuestion, maxAgents: 3);
    
    Console.WriteLine("\n" + new string('=', 60));
    Console.WriteLine("GROUP CHAT COMPLETED SUCCESSFULLY!");
    Console.WriteLine(new string('=', 60));
    Console.WriteLine($"Complex question answered using multiple expert perspectives!");
    Console.WriteLine($"Timestamp: {DateTime.Now:yyyy-MM-dd HH:mm:ss}");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error in group chat: {ex.Message}");
    Console.WriteLine("\nTroubleshooting tips:");
    Console.WriteLine("1. Check your Azure OpenAI configuration");
    Console.WriteLine("2. Verify all agents are properly initialized");
}

### Custom Group Chat Question

In [ ]:
// Try Your Own Group Chat Question
// Modify the question below to test different scenarios

Console.WriteLine("Testing Group Chat with Your Custom Question");
Console.WriteLine("Feel free to modify the question below to explore different topics!");
Console.WriteLine();

// MODIFY THIS QUESTION to test different scenarios:
var customQuestion = @"How can we improve team productivity in a remote software development team? 
Consider technical tools, management practices, and team culture.";

Console.WriteLine($"Your Question: {customQuestion}");
Console.WriteLine(new string('=', 60));

try
{
    // Run group chat with your question
    var result = await orchestrator.GroupChatAsync(customQuestion, maxAgents: 3);
    
    Console.WriteLine("\n" + new string('=', 60));
    Console.WriteLine("YOUR CUSTOM GROUP CHAT COMPLETED!");
    Console.WriteLine(new string('=', 60));
    Console.WriteLine("Agents provided comprehensive insights from multiple perspectives!");
    Console.WriteLine($"Timestamp: {DateTime.Now:yyyy-MM-dd HH:mm:ss}");
    
    Console.WriteLine();
    Console.WriteLine("Feel free to modify the customQuestion above to explore different topics!");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error in custom group chat: {ex.Message}");
}

## Integration with .NET API Backend

This workshop connects with the actual .NET Semantic Kernel API backend. The agents you've created here follow the same patterns used in the production API.

### Key Integration Points:
1. **Agent Architecture** - Uses the same `ChatCompletionAgent` pattern as the backend
2. **Group Chat System** - Implements `AgentGroupChat` for multi-agent coordination  
3. **Kernel Configuration** - Same Azure OpenAI setup as the API
4. **Agent Instructions** - Compatible instruction patterns for consistent behavior

### Production Features Available:
- **REST API Endpoints** - `/group-chat` for multi-agent conversations
- **Session Management** - Persistent conversation history
- **Agent Registry** - Dynamic agent management and selection
- **Health Monitoring** - System health and performance tracking
- **Error Handling** - Robust error handling and logging

### Try the API:
You can test the running API backend at `http://localhost:5000` (if running) with endpoints like:
- `GET /health` - Check system health
- `GET /agents` - List available agents  
- `POST /group-chat` - Start multi-agent conversations

The notebook agents you've created demonstrate the core capabilities that power the full API system!